# Constructing SQLite Tables for Notebooks and Search

SQLite full text search setup via APSW for all the notebooks on this website, inspired by the [APSW FTS5 Tour]((https://rogerbinns.github.io/apsw/example-fts.html)).

## Setup

In [9]:
from typing import Optional, Iterator, Any

from pprint import pprint
import re
import functools

import apsw
import apsw.ext
import apsw.fts5
import apsw.fts5aux
import apsw.fts5query

from execnb.nbio import read_nb
from fastcore.all import *
from pathlib import Path, PosixPath

In [3]:
print("FTS5 available:", "ENABLE_FTS5" in apsw.compile_options)

FTS5 available: True


## Create a Notebooks Database

Until now I haven't had a SQLite database for anything on this site. Let's create one.

In [5]:
connection = apsw.Connection("notebooks.db")
connection

## Create and Populate Table `notebooks`

In [36]:
nbs = L(Path("../arg-blog-fasthtml/nbs").glob("*.ipynb")).sorted(reverse=True)
nbs

(#35) [Path('../arg-blog-fasthtml/nbs/2025-01-12-A-Better-Notebook-Index-Page.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-11-NBClassic-Keyboard-Shortcuts-in-Command-and-Dual-Mode.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-10-Understanding-FastHTML-Routes-Requests-and-Redirects.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-09-Reading-and-Writing-Jupyter-Notebooks-With-Python.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-08-HTML-Title-Tag-in-FastHTML.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-07-Verifying-Bluesky-Domain-in-FastHTML.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-06-Understanding-FastHTML-Headers.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-05-SSH-Agent-to-Save-Passphrase-Typing.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-04-Claude-Artifacts-in-Notebooks.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-03-Using-zip.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-02-FastHTML-Piano-Part-3.ipynb'),Path('../arg-blog-fasthtml/nbs/2025-01-02-FastHTML-Piano-Part-2.

All notebooks, sorted from newest to oldest.

In [38]:
nb = read_nb(nbs[0])
nb.cells[0]

```json
{ 'cell_type': 'markdown',
  'id': 'b8ca3733',
  'idx_': 0,
  'metadata': {},
  'source': '# A Better Notebook Index Page'}
```

A Markdown cell looks like this.

In [39]:
connection.execute("""CREATE TABLE IF NOT EXISTS notebooks (
    id INTEGER PRIMARY KEY,
    path TEXT NOT NULL,
    markdown_content TEXT)""")

We create a table to put notebooks' paths and Markdown content into. (At this point we skip code cells to make things simple.)

In [41]:
def is_md_cell(c): return c.cell_type == 'markdown'
md_cells = L(nb.cells).filter(is_md_cell)
md_cells

(#39) [{'cell_type': 'markdown', 'id': 'b8ca3733', 'metadata': {}, 'source': '# A Better Notebook Index Page', 'idx_': 0},{'cell_type': 'markdown', 'id': '0adccd3d', 'metadata': {}, 'source': "I've made good progress on creating a notebook every day. Now I have so many notebooks that my index page needs an overhaul, including:\n\n* Dates with datetime\n* Cards with execnb to grab notebook titles\n* The cache decorator to make that fast\n* Subtle CSS tweaks to increase information density", 'idx_': 1},{'cell_type': 'markdown', 'id': '9f9ab0bb', 'metadata': {}, 'source': '## List Live Posts', 'idx_': 3},{'cell_type': 'markdown', 'id': '8e35e71e', 'metadata': {}, 'source': 'According to this, I have 34 notebooks in `arg-blog-fasthtml/nbs`, which matches the 34 cards on audrey.feldroy.com.', 'idx_': 5},{'cell_type': 'markdown', 'id': '506c2763', 'metadata': {}, 'source': '## Pathlib, User Directory, and PosixPath', 'idx_': 6},{'cell_type': 'markdown', 'id': 'c57a8f71', 'metadata': {}, 'sou

A list of only Markdown cells from a notebook looks like this.

In [42]:
def cell_source(c): return c.source
md = md_cells.map(cell_source)
md

(#39) ['# A Better Notebook Index Page',"I've made good progress on creating a notebook every day. Now I have so many notebooks that my index page needs an overhaul, including:\n\n* Dates with datetime\n* Cards with execnb to grab notebook titles\n* The cache decorator to make that fast\n* Subtle CSS tweaks to increase information density",'## List Live Posts','According to this, I have 34 notebooks in `arg-blog-fasthtml/nbs`, which matches the 34 cards on audrey.feldroy.com.','## Pathlib, User Directory, and PosixPath','To specify the path in terms of my home directory `~`, I use `PosixPath`.','Here I expand `~` into `/Users/arg/`, list files that end in .ipynb, and convert the generator object into a readable list with a fastcore `L` list.','## Display the Notebooks List Nicely','If we just print the filenames, we can see my current approach of naming them with the date and TitleCase title.','## New Approach','Doing this has given me insight about how to improve my site:\n\n* Keep na

Map the cells to just their Markdown source. For now we don't care about the rest.

In [46]:
def extract_markdown_content(nbpath):
    """Extract all markdown cell content from a notebook"""
    nb = read_nb(nbpath)
    md_cells = L(nb.cells).filter(is_md_cell)
    return "\n".join(md_cells.map(cell_source))
extract_markdown_content(nbs[0])

"# A Better Notebook Index Page\nI've made good progress on creating a notebook every day. Now I have so many notebooks that my index page needs an overhaul, including:\n\n* Dates with datetime\n* Cards with execnb to grab notebook titles\n* The cache decorator to make that fast\n* Subtle CSS tweaks to increase information density\n## List Live Posts\nAccording to this, I have 34 notebooks in `arg-blog-fasthtml/nbs`, which matches the 34 cards on audrey.feldroy.com.\n## Pathlib, User Directory, and PosixPath\nTo specify the path in terms of my home directory `~`, I use `PosixPath`.\nHere I expand `~` into `/Users/arg/`, list files that end in .ipynb, and convert the generator object into a readable list with a fastcore `L` list.\n## Display the Notebooks List Nicely\nIf we just print the filenames, we can see my current approach of naming them with the date and TitleCase title.\n## New Approach\nDoing this has given me insight about how to improve my site:\n\n* Keep naming files as bef

Join all the Markdown cells' content for a notebook together, separated by 1 new line between each pair of cells.

In [47]:
def populate_notebooks_table():
    connection.execute("DELETE FROM notebooks")
    
    for nb_path in nbs:
        markdown_text = extract_markdown_content(nb_path)
        connection.execute(
            "INSERT INTO notebooks (path, markdown_content) VALUES (?, ?)",
            (str(nb_path), markdown_text)
        )
populate_notebooks_table()

In [59]:
connection.execute("SELECT count(*) FROM notebooks").get

35

Now we have notebook paths and their contents in a SQLite table.

## Create Search Table

In [52]:
if not connection.table_exists("main", "search"):
    search_table = apsw.fts5.Table.create(
        connection,
        "search",
        content="notebooks",
        columns=None,
        generate_triggers=True,
        tokenize=["simplify","casefold","true","strip","true","unicodewords"])
else:
    search_table = apsw.fts5.Table(connection, "search")

The SQLite FTS5 extension has you create a virtual search table, where you point it at the name of the table containing the content to search.

In [53]:
print("quoted name", search_table.quoted_table_name)

quoted name "main"."search"


The notebooks' content:

In [61]:
print(connection.execute(
        "SELECT sql FROM sqlite_schema WHERE name='notebooks'"
    ).get)

CREATE TABLE notebooks (
    id INTEGER PRIMARY KEY,
    path TEXT NOT NULL,
    markdown_content TEXT)


In [62]:
pprint(search_table.structure)

FTS5TableStructure(name='search',
                   columns=('id', 'path', 'markdown_content'),
                   unindexed=set(),
                   tokenize=('simplify',
                             'casefold',
                             'true',
                             'strip',
                             'true',
                             'unicodewords'),
                   prefix=set(),
                   content='notebooks',
                   content_rowid='_ROWID_',
                   contentless_delete=None,
                   contentless_unindexed=None,
                   columnsize=True,
                   tokendata=False,
                   locale=False,
                   detail='full')


In [63]:
print(f"{search_table.config_rank()=}")

search_table.config_rank()='bm25()'


In [64]:
print(f"{search_table.row_count=}")

search_table.row_count=35


In [65]:
print(f"{search_table.tokens_per_column=}")

search_table.tokens_per_column=[35, 373, 8700]


To be continued...